In [13]:
from napkin import response, request
from pymongo import MongoClient
import requests
import os
import string
import random

CLIENT_NAME = request.path_params.get("CLIENT_NAME")
MONGO_AUTH = os.environ.get("MONGO_AUTH")
GAPP_KEY = os.environ.get("GAPP_KEY")

APP_URL = "https://script.google.com/macros/s/"
EMAIL = request.path_params.get("EMAIL")

In [14]:
client = MongoClient("mongodb+srv://{}@meusite.hcxhp.mongodb.net/Site?retryWrites=true&w=majority".format(MONGO_AUTH))
db = client.Site

Coll = db[CLIENT_NAME]
Auth = Coll.find_one({ "_id": "Auth", "users.email": EMAIL })

In [18]:
from passlib.context import CryptContext

pwd_context = CryptContext(
  schemes=["pbkdf2_sha256"],
  default="pbkdf2_sha256",
  pbkdf2_sha256__default_rounds=30000
)

'$pbkdf2-sha256$30000$qHVurTXmnBNirBViDOE8xw$FzJtIfa11B4pePZOH70o3DwcjmkztuLeaOAAP3P7uAg'

In [16]:
if Auth != None:
  password = ''.join(random.sample(string.ascii_letters, 6))
  
  users = Auth.get("users")
  index, user = [(i, usr) for (i, usr) in enumerate(users) if usr.get("email") == EMAIL][0]

  user['pass'] = pwd_context.hash(password)
  users[index] = user
  Coll.update_one({ "_id": "Auth" }, { "$set": { "users": users } })

  try:
    email = {
      "to": EMAIL,
      "subject": "Alteração de senha",
      "message": "Sua nova senha é {}".format(password)
    }
    data = requests.get(APP_URL + GAPP_KEY, params=email)
    email_response = data.json()
    print(email_response)

    if email_response["status"] == "success":
      response.status_code = 200
    else:
      response.status_code = 400
      response.body = email_response.message
  except:
    response.status_code = 400
    response.body = "Offline"


{'status': 'success'}


NameError: name 'response' is not defined